In [4]:
## Have to run the baseline on test set and trained network on test set 
## What is my test set? Which video kept aside 
## h5_new_data.py - train_one_aside.h5 and test_one_aside.h5
## After merging with old data too - complete_train_one_aside.h5 and complete_test_one_aside.h5
## scenes[1] is kept aside 

## Imports 
import pdb 
from path import Path 
import scipy.io 
import os 
import PIL.Image as Image
import cv2
import numpy as np 
import h5py 
import matplotlib.pyplot as plt 
from scipy.ndimage.filters import gaussian_filter 
import hdf5storage

In [5]:
## Paths to data, collected in november using camera and LiDAR 
root = Path('/mnt/hdd1/aashi/cmu_data/left_imgs_nov_2')
label_dir_left = Path('/mnt/hdd1/aashi/cmu_data/labels_left_2/')
det_dir_left = Path('/mnt/hdd1/aashi/cmu_data/det_left_2/')
label_dir_right = Path('/mnt/hdd1/aashi/cmu_data/labels_right_2/')
det_dir_right = Path('/mnt/hdd1/aashi/cmu_data/det_right_2')
scenes = root.dirs()
## Detections saved in det_dir_left and det_dir_right 

In [10]:
## Naive baseline - no sequence of images
## (1) See the current image (2) Read detections 
## (3) Select the required coordinate (4) Put threshold 

thresh = 400 

for no, scene in enumerate(scenes):
    
    if no == 1:
        name = os.path.split(scene)[-1]
        mat = hdf5storage.loadmat('/mnt/hdd1/aashi/cmu_data/mats_nov_2/'
                                 + str(name) + '.mat')
        left_imgs = mat['left_imgs']
#         pdb.set_trace()
        arr_left = np.loadtxt(label_dir_left + 'labels_' + name + '.txt')
        ## Gaussian smoothing over labels 
        arr_left = gaussian_filter(arr_left, sigma=0.5)
#         arr_left = arr_left > 1e-8
        num = arr_left.shape[0]
        
        ## Record the confusion matrix 
        confusion_matrix = np.zeros((2,2))
        #sequence_len = 10
        sequence_len = 20 
        for i in range(0, num - sequence_len):
            ## leftImg = cv2.imread(scene + '/' + str(i+1) + '.png') 
            detFile = det_dir_left + 'det_' + str(name) + '/' + str(i+1) + '.txt'
            pred_label = 0
            if (os.path.getsize(detFile)):
                ## Read the bounding box 
                bboxes = np.loadtxt(detFile)
                ## img = left_imgs[0][i]
                if (len(bboxes.shape) == 1):
                    x1 = int(bboxes[0])
                    y1 = int(bboxes[1])
                    x2 = int(bboxes[2])
                    y2 = int(bboxes[3])
                    if (y2 > thresh):
                        pred_label = 1
                else:
                    for k in range(bboxes.shape[0]): 
                    ## Loop over each pedestrian
                        x1 = int(bboxes[k][0])
                        y1 = int(bboxes[k][1])
                        x2 = int(bboxes[k][2])
                        y2 = int(bboxes[k][3])
                        ## cv2.rectangle(leftImg, (x1, y1), (x2, y2), (255, 255, 0), 2)
                        ## last element is probability - don't care
                        if (y2 > thresh):
                            pred_label = 1
                    ## pdb.set_trace()
                
            true_label = int(sum(arr_left[i:i+sequence_len]) > 0.1)
            confusion_matrix[pred_label][true_label] += 1 
            
                

In [11]:
confusion_matrix


array([[272.,  24.],
       [375., 465.]])

In [12]:
tp = confusion_matrix[1][1]
fp = confusion_matrix[1][0]
fn = confusion_matrix[0][1]
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1Score = 2*precision*recall/(precision + recall)

In [13]:
f1Score

0.6997742663656885

In [7]:
## Now, the naive baseline and OurApproach our comparable 
## Both predict if 1 second in future on complete video (pedestrian or not)
## OurApproach: 0.7233 and naive baseline: 0.58 (at threshold of 500 on 720 image)
## OurApproach: 0.7233 and naive baseline: 0.711 (at threshold of 450 on 720 image)

## Put the data collected today and see if the performance remains same 

root = Path('/mnt/hdd1/aashi/cmu_data/left_imgs_dec')
label_dir_left = Path('/mnt/hdd1/aashi/cmu_data/labels_left_dec/')
det_dir_left = Path('/mnt/hdd1/aashi/cmu_data/det_left_dec/')
scenes = root.dirs()

for no, scene in enumerate(scenes):
    
        name = os.path.split(scene)[-1]
        mat = scipy.io.loadmat('/mnt/hdd1/aashi/cmu_data/mats_dec/'
                                 + str(name) + '.mat')
        left_imgs = mat['left_imgs']
        arr_left = np.loadtxt(label_dir_left + 'labels_' + name + '.txt')
        ## Gaussian smoothing over labels 
        arr_left = gaussian_filter(arr_left, sigma=0.5)
#         arr_left = arr_left > 1e-8
        num = arr_left.shape[0]
        
        ## Record the confusion matrix 
#         confusion_matrix = np.zeros((2,2))
        sequence_len = 10
        for i in range(0, num - sequence_len):
            ## leftImg = imread(scene + '/' + str(i+1) + '.png') 
            detFile = det_dir_left + 'det_' + str(name) + '/' + str(i+1) + '.txt'
            pred_label = 0
            if (os.path.getsize(detFile)):
                ## Read the bounding box 
                bboxes = np.loadtxt(detFile)
                ## img = left_imgs[0][i]
                if (len(bboxes.shape) == 1):
                    x1 = int(bboxes[0])
                    y1 = int(bboxes[1])
                    x2 = int(bboxes[2])
                    y2 = int(bboxes[3])
                    if (y2 > thresh):
                        pred_label = 1
                else:
                    for k in range(bboxes.shape[0]): 
                    ## Loop over each pedestrian
                        x1 = int(bboxes[k][0])
                        y1 = int(bboxes[k][1])
                        x2 = int(bboxes[k][2])
                        y2 = int(bboxes[k][3])
                        ## cv2.rectangle(leftImg, (x1, y1), (x2, y2), (255, 255, 0), 2)
                        ## last element is probability - don't care
                        if (y2 > thresh):
                            pred_label = 1
                    ## pdb.set_trace()
                
            true_label = int(sum(arr_left[i:i+sequence_len]) > 0.1)
            confusion_matrix[pred_label][true_label] += 1 

In [8]:
confusion_matrix

array([[1804.,  159.],
       [ 151.,  359.]])

In [9]:
tp = confusion_matrix[1][1]
fp = confusion_matrix[1][0]
fn = confusion_matrix[0][1]
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1Score = 2*precision*recall/(precision + recall)

In [10]:
print(f1Score)

0.698443579766537


In [11]:
## After adding new data: 
## OurApproach: 0.696, Baseline: 0.69844; 